In [ ]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pickle

In [ ]:
#https://nha.chotot.com/thue-phong-tro-da-nang?page=1
#https://phongtro123.com/tinh-thanh/da-nang

In [ ]:
drive = webdriver.Chrome(ChromeDriverManager().install())
drive.maximize_window()
wait = WebDriverWait(drive, 10)

In [ ]:
page = 1
house_infos = list()
while True:
    try:
        drive.get('https://nha.chotot.com/thue-phong-tro-da-nang?page=' + str(page))
    except: break
    houses = drive.find_elements(By.XPATH, value = '//a[@href and @class="AdItem_adItem__2O28x"]')
    if (houses == []) break;
    for i in range(len(houses)):
        houses[i] = houses[i].get_attribute('href')
    for house in houses:
        drive.get(house)
        try:
            house_location= drive.find_element(By.XPATH, value = '//span[@class="fz13"]').text.replace('Xem bản đồ','')
            house_price = drive.find_element(By.XPATH, value = '//span[@itemprop="price"]').text
            house_area = drive.find_element(By.XPATH, value = '//span[@class="AdDecription_squareMetre__2KYh8"]').text
            house_name = drive.find_element(By.XPATH, value = '//h1[@class="AdDecription_adTitle__2I0VE" and @itemprop="name"]').text
            house_date = drive.find_element(By.XPATH, value = '//span[@class="AdImage_imageCaptionText__39oDK"]').text
            house_features = drive.find_elements(By.XPATH, value = '//div[@class="col-md-6 no-padding AdParam_adParamItem__1o-dd" and @data-testid="param-item"]')
            house_info = {}
            house_info['Location'] = house_location
            house_info['Price'] = house_price
            house_info['Area'] = house_area
            house_info['Name'] = house_name
            house_info['Date'] = house_date
            for index in range(len(house_features)):
                if house_features[index].text[0] == 'T': house_info['Feature' + str(2)] = house_features[index].text
                elif house_features[index].text[0] == 'S': house_info['Feature' + str(3)] = house_features[index].text
                else: house_info['Feature' + str(index)] = house_features[index].text
            house_infos.append(house_info)
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('save_house.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
with open('save_house.pickle','wb') as handle:
    pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
drive.close()

In [ ]:
dataset = pd.DataFrame(data = house_infos)
dataset.head()

In [ ]:
dataset.to_csv('raw_data.csv')